<a href="https://colab.research.google.com/github/hwangho-kim/Utility-OAC/blob/main/Daily_FDC_Monitoring_with_Fixed_Reference_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
import joblib
from datetime import date, timedelta

# --- 그래프 한글 폰트 설정 ---
try:
    import koreanize_matplotlib
except ImportError:
    pass

# --- 데이터 생성 함수들 ---
def create_long_format_csv(filepath, num_wafers, anomaly_info=None):
    """
    지정된 조건으로 Long Format CSV 데이터를 생성하는 범용 함수
    """
    print(f"'{filepath}' 이름으로 샘플 CSV 파일을 생성합니다...")
    num_steps = 10
    num_sensors = 10
    time_points = 10
    np.random.seed(42)

    records = []
    for wafer_id in range(1, num_wafers + 1):
        for step_id in range(1, num_steps + 1):
            record_base = {'wafer_id': wafer_id, 'step_id': step_id}
            for i in range(num_sensors):
                sensor_name = f'Sensor_{chr(65+i)}'
                record_base[sensor_name] = np.random.randn(time_points) * (i * 0.1 + 0.5) + (i * 5 + step_id)

            # 이상 상태 또는 열화 주입
            if anomaly_info:
                if (anomaly_info.get('wafer_id') is None or wafer_id >= anomaly_info['wafer_id']) and \
                   (anomaly_info.get('step_id') is None or step_id == anomaly_info['step_id']):
                    record_base['Sensor_C'] += anomaly_info.get('c_drift', 0)
                    record_base['Sensor_G'] *= anomaly_info.get('g_noise', 1)

            for t_idx in range(time_points):
                row = {'wafer_id': record_base['wafer_id'], 'step_id': record_base['step_id'], 'time': t_idx}
                for i in range(num_sensors):
                    sensor_name = f'Sensor_{chr(65+i)}'
                    row[sensor_name] = record_base[sensor_name][t_idx]
                records.append(row)

    df = pd.DataFrame(records)
    df.to_csv(filepath, index=False)
    print("샘플 파일 생성 완료.")
    return df

# --- 특징 추출 및 컬럼 탐지 함수들 (이전과 동일) ---
def find_columns(df):
    id_candidates = {'wafer': ['wafer_id'], 'step': ['step_id'], 'time': ['time']}
    detected_cols = {}
    remaining_cols = list(df.columns)
    for id_type, candidates in id_candidates.items():
        found = False
        for col in remaining_cols:
            if col.lower() in candidates:
                detected_cols[id_type] = col; remaining_cols.remove(col); found = True; break
        if not found: detected_cols[id_type] = None
    sensor_cols = [col for col in remaining_cols if pd.api.types.is_numeric_dtype(df[col])]
    return detected_cols['wafer'], detected_cols['step'], sensor_cols

def extract_and_pivot_features(df, wafer_id_col, step_id_col, sensor_cols):
    def rms(x): return np.sqrt(np.mean(x**2))
    agg_funcs = ['mean', 'std', 'max', 'min', 'median', 'skew', pd.Series.kurt, rms]
    features = df.groupby([wafer_id_col, step_id_col])[sensor_cols].agg(agg_funcs).reset_index()
    new_cols = [wafer_id_col, step_id_col]
    for col_level0, col_level1 in features.columns[2:]:
        func_name = col_level1 if isinstance(col_level1, str) else col_level1.__name__
        new_cols.append(f"{col_level0}_{func_name}")
    features.columns = new_cols
    feature_pivot = features.pivot(index=wafer_id_col, columns=step_id_col)
    feature_pivot.columns = [f"S{int(col[1])}_{col[0]}" for col in feature_pivot.columns.values]
    feature_pivot.fillna(0, inplace=True)
    return feature_pivot.reset_index()

# ======================================================================================
# Phase 1: 기준 모델 생성 (최초 1회 실행)
# ======================================================================================
def train_and_save_reference_model(golden_data_path='golden_data.csv'):
    print("\n" + "="*25 + " Phase 1: 기준 모델 생성 " + "="*25)

    # 1. 골든 데이터셋 생성
    create_long_format_csv(golden_data_path, num_wafers=100)
    df_long = pd.read_csv(golden_data_path)

    # 2. 특징 추출 및 변환
    wafer_id_col, step_id_col, sensor_cols = find_columns(df_long)
    df_wide = extract_and_pivot_features(df_long, wafer_id_col, step_id_col, sensor_cols)
    feature_cols = [col for col in df_wide.columns if col != wafer_id_col]

    # 3. DBSCAN으로 정상 군집 찾기
    scaler_dbscan = StandardScaler()
    X_scaled = scaler_dbscan.fit_transform(df_wide[feature_cols])

    try: from kneed import KneeLocator
    except ImportError: print("오류: 'kneed' 라이브러리가 필요합니다. 'pip install kneed'를 실행해주세요."); return

    nearest_neighbors = NearestNeighbors(n_neighbors=5).fit(X_scaled)
    distances, _ = nearest_neighbors.kneighbors(X_scaled)
    sorted_distances = np.sort(distances[:, 4])
    kneedle = KneeLocator(x=range(1, len(sorted_distances)+1), y=sorted_distances, S=1.0, curve="convex", direction="increasing")
    optimal_eps = kneedle.elbow_y or np.median(sorted_distances)

    dbscan = DBSCAN(eps=optimal_eps, min_samples=5)
    clusters = dbscan.fit_predict(X_scaled)
    df_wide['dbscan_cluster'] = clusters

    main_cluster_label = pd.Series(clusters).value_counts().idxmax()
    golden_df = df_wide[df_wide['dbscan_cluster'] == main_cluster_label].copy()
    print(f"골든 데이터셋에서 {len(golden_df)}개의 정상 Wafer를 식별했습니다.")

    # 4. 정상 군집 데이터로 기준 모델(스케일러, PCA) 학습
    scaler_pca = StandardScaler()
    golden_features_scaled = scaler_pca.fit_transform(golden_df[feature_cols])

    pca_model = PCA(n_components=0.95)
    pca_model.fit(golden_features_scaled)

    # 5. 학습된 모델 파일로 저장
    joblib.dump(scaler_pca, 'scaler.joblib')
    joblib.dump(pca_model, 'pca.joblib')
    joblib.dump(feature_cols, 'feature_cols.joblib') # 특징 이름 순서도 저장

    print("\n기준 모델 생성 및 저장 완료: scaler.joblib, pca.joblib, feature_cols.joblib")
    print("="*70)

# ======================================================================================
# Phase 2: 일일 모니터링 (매일 반복 실행)
# ======================================================================================
def run_daily_monitoring(days_to_monitor=10):
    print("\n" + "="*25 + " Phase 2: 일일 모니터링 시작 " + "="*25)

    # 1. 저장된 기준 모델 불러오기
    try:
        scaler_pca = joblib.load('scaler.joblib')
        pca_model = joblib.load('pca.joblib')
        feature_cols = joblib.load('feature_cols.joblib')
        print("저장된 기준 모델(scaler, pca, feature_cols)을 성공적으로 불러왔습니다.")
    except FileNotFoundError:
        print("오류: 기준 모델 파일(.joblib)을 찾을 수 없습니다. Phase 1을 먼저 실행해주세요.")
        return

    daily_log = []
    start_date = date.today()

    for day in range(days_to_monitor):
        current_date = start_date + timedelta(days=day)
        print(f"\n--- {current_date} 데이터 모니터링 ---")

        # 2. 일일 데이터 생성 (시간이 지남에 따라 미세하게 열화)
        daily_data_path = f'daily_data_day_{day+1}.csv'
        degradation_info = {'c_drift': day * 0.1, 'g_noise': 1 + day * 0.02}
        create_long_format_csv(daily_data_path, num_wafers=20, anomaly_info=degradation_info)
        df_long_daily = pd.read_csv(daily_data_path)

        # 3. 특징 추출 및 변환
        wafer_id_col, step_id_col, sensor_cols = find_columns(df_long_daily)
        df_wide_daily = extract_and_pivot_features(df_long_daily, wafer_id_col, step_id_col, sensor_cols)

        # 저장된 특징 순서와 동일하게 맞춤
        df_wide_daily = df_wide_daily[df_wide_daily.columns.intersection([wafer_id_col] + feature_cols)]

        # 4. 불러온 기준 모델로 Health Index 계산 (절대 학습(fit)하지 않음!)
        daily_features_scaled = scaler_pca.transform(df_wide_daily[feature_cols])
        daily_pca_scores = pca_model.transform(daily_features_scaled)

        health_indexes = np.abs(daily_pca_scores[:, 0])

        # 5. Daily Index 계산 및 기록
        daily_index = np.mean(health_indexes)
        print(f"{current_date}의 Daily Index: {daily_index:.4f}")
        daily_log.append({'date': current_date, 'daily_index': daily_index})

    # 6. Daily Index 로그 저장 및 시각화
    log_df = pd.DataFrame(daily_log)
    log_df.to_csv('daily_health_log.csv', index=False)
    print("\n일일 모니터링 로그 저장 완료: daily_health_log.csv")

    plt.figure(figsize=(12, 6))
    sns.lineplot(x='date', y='daily_index', data=log_df, marker='o')
    plt.title('Daily Equipment Health Index Trend', fontsize=16)
    plt.xlabel('Date')
    plt.ylabel('Daily Health Index (Lower is Better)')
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--')
    plt.tight_layout()
    plt.show()


if __name__ == '__main__':
    # Phase 1: 기준이 되는 모델을 생성하고 파일로 저장합니다.
    # 이 부분은 실제 운영 시, 장비가 가장 안정적일 때 단 한 번만 실행합니다.
    train_and_save_reference_model()

    # Phase 2: 저장된 모델을 불러와 매일 모니터링을 수행합니다.
    # 이 부분은 매일 새로운 데이터에 대해 반복적으로 실행됩니다.
    run_daily_monitoring(days_to_monitor=10)